lazypredict https://pypi.org/project/lazypredict/

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pandas as pd

In [2]:
train = pd.read_csv('../Data/train.csv')
test = pd.read_csv('../Data/test.csv')

In [4]:
from sklearn import *
import sklearn as sklearn
chosen_regressors = [('ARDRegression', sklearn.linear_model._bayes.ARDRegression),
 ('AdaBoostRegressor', sklearn.ensemble._weight_boosting.AdaBoostRegressor),
 ('BaggingRegressor', sklearn.ensemble._bagging.BaggingRegressor),
 ('BayesianRidge', sklearn.linear_model._bayes.BayesianRidge),
 ('CCA', sklearn.cross_decomposition._pls.CCA),
 ('DecisionTreeRegressor', sklearn.tree._classes.DecisionTreeRegressor),
 ('DummyRegressor', sklearn.dummy.DummyRegressor),
 ('ElasticNet', sklearn.linear_model._coordinate_descent.ElasticNet),
 ('ElasticNetCV', sklearn.linear_model._coordinate_descent.ElasticNetCV),
 ('ExtraTreeRegressor', sklearn.tree._classes.ExtraTreeRegressor),
 ('ExtraTreesRegressor', sklearn.ensemble._forest.ExtraTreesRegressor),
 ('GradientBoostingRegressor', sklearn.ensemble._gb.GradientBoostingRegressor),
 ('HistGradientBoostingRegressor',
  sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingRegressor),
 ('HuberRegressor', sklearn.linear_model._huber.HuberRegressor),
 ('KNeighborsRegressor', sklearn.neighbors._regression.KNeighborsRegressor),
 ('Lars', sklearn.linear_model._least_angle.Lars),
 ('LarsCV', sklearn.linear_model._least_angle.LarsCV),
 ('Lasso', sklearn.linear_model._coordinate_descent.Lasso),
 ('LassoCV', sklearn.linear_model._coordinate_descent.LassoCV),
 ('LassoLars', sklearn.linear_model._least_angle.LassoLars),
 ('LassoLarsCV', sklearn.linear_model._least_angle.LassoLarsCV),
 ('LassoLarsIC', sklearn.linear_model._least_angle.LassoLarsIC),
 ('LinearRegression', sklearn.linear_model._base.LinearRegression),
 ('LinearSVR', sklearn.svm._classes.LinearSVR),
 ('MLPRegressor', sklearn.neural_network._multilayer_perceptron.MLPRegressor),
 ('OrthogonalMatchingPursuit',
  sklearn.linear_model._omp.OrthogonalMatchingPursuit),
 ('OrthogonalMatchingPursuitCV',
  sklearn.linear_model._omp.OrthogonalMatchingPursuitCV),
 ('PassiveAggressiveRegressor',
  sklearn.linear_model._passive_aggressive.PassiveAggressiveRegressor),
 ('PoissonRegressor', sklearn.linear_model._glm.glm.PoissonRegressor),
 ('RANSACRegressor', sklearn.linear_model._ransac.RANSACRegressor),
 ('RandomForestRegressor', sklearn.ensemble._forest.RandomForestRegressor),
 ('Ridge', sklearn.linear_model._ridge.Ridge),
 ('RidgeCV', sklearn.linear_model._ridge.RidgeCV),
 ('SGDRegressor', sklearn.linear_model._stochastic_gradient.SGDRegressor),
 ('SVR', sklearn.svm._classes.SVR),
 ('TheilSenRegressor', sklearn.linear_model._theil_sen.TheilSenRegressor),
 ('TransformedTargetRegressor',
  sklearn.compose._target.TransformedTargetRegressor),
 ('TweedieRegressor', sklearn.linear_model._glm.glm.TweedieRegressor)]

In [5]:
X_train = train.drop('ttf', axis=1)
y_train = train['ttf']
X_test = test.drop('ttf', axis=1)
y_test = test['ttf']

In [6]:
import numpy as np
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)

In [7]:
reg.score(X_test, y_test)

0.6590268440608174

In [8]:
import lazypredict
from lazypredict.Supervised import LazyRegressor

reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None, regressors=chosen_regressors)
models, pred = reg.fit(X_train, X_test, y_train, y_test)

models

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|███████████████████████████████████████████| 38/38 [00:39<00:00,  1.03s/it]


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
GradientBoostingRegressor,0.73,0.73,0.10,3.68
HistGradientBoostingRegressor,0.73,0.73,0.10,0.33
ExtraTreesRegressor,0.73,0.73,0.10,4.75
RandomForestRegressor,0.72,0.72,0.10,13.99
MLPRegressor,0.71,0.71,0.10,0.90
SVR,0.71,0.71,0.10,6.60
BaggingRegressor,0.70,0.70,0.10,1.24
KNeighborsRegressor,0.67,0.67,0.11,0.82
LassoCV,0.66,0.66,0.11,0.12
